In [1]:
import pandas as pd
import time
from google import genai
from data import read_data
from datetime import datetime
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

In [ ]:
# data = pd.read_csv("smile-college-dataset.csv")
# data.drop(data.columns[0], axis=1, inplace=True)
data =read_data()
data

In [ ]:
df = data.copy()
count = 0
batch_size = 10
start_index = 0
n_rows = 100 # how many rows to traverse, if you dont want to run whole df
max_rows = min(start_index+n_rows, len(df))

while start_index < max_rows:
    end_index = min(start_index+batch_size, max_rows)
    batch_df = df.iloc[start_index:end_index]
    
    client = genai.Client()

    print(f"Processing rows {start_index} to {end_index - 1}")
    for index, row in batch_df.iterrows():
        expected_output = row["Validated_Labels"]
        query = row["comment"]
        prompt = f"""
            You are an experienced sentiment analyst studying students’ feedback on mental health services.
            Categorize the following response into one of these four labels: Satisfied, Dissatisfied, Mixed, Neutral.

            The specific criteria for each category are as follows. 
            (1) “Satisfied”: at least 75% of the language expressed satisfaction, with minimal suggestions for improvement. 
            (2) “Dissatisfied”: at least 75% of the language indicated discontent or suggestions for enhancement, with little mention of satisfaction. 
            (3) “Mixed”: expressions of satisfaction and dissatisfaction/suggestions were approximately evenly split, with each constituting about 50%. 
            (4) “Neutral”: no clear emphasis on satisfaction, dissatisfaction, or suggestions for improvement

            Question: “What mental health or wellness services and supports provided by your college are working well? What aspects need more attention?”
            Response: {query}

            Output: 
            """

        response = client.models.generate_content(
            model="gemini-2.5-flash", contents=prompt
        )
        response = response.text.strip().lower()
        df.loc[index, "Predicted_labels"] = response
        print(f"{index}. Model Output: {response}, Actual label = {expected_output}")
        
        # print to csv
    df[start_index:end_index].to_csv(f"partial_{start_index}_{end_index} - {datetime.now()}.csv")


    print("Resetting connection...\n")
    client.close()   
    time.sleep(60) 
    start_index = end_index
        
    

In [ ]:
df.to_csv(f"Output-{datetime.now()}.csv")